<a href="https://colab.research.google.com/github/Sameer-30/Deflection-of-a-beam-using-finite-elements/blob/main/Beam_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Beam Deflection Analysis for Clamped-Clamped Beam Under Uniform Load

## Question Statement

A uniform Euler-Bernoulli beam of length 1 meter is made of steel with the following properties:

- **Young’s Modulus (E)**: \( 200 \times 10^9 \, \text{Pa} \)
- **Density (ρ)**: \( 7800 \, \text{kg/m}^3 \)
- **Cross-section**:
  - **Width (b)**: 10 mm
  - **Height (h)**: 5 mm
  - **Moment of Inertia (I)**:
  \[
  I = \frac{b h^3}{12}
  \]
  - **Cross-sectional Area (A)**:
  \[
  A = b h
  \]

The beam is divided into 40 finite elements for analysis.

### Boundary Condition
The beam is clamped at both ends (clamped-clamped condition), meaning zero displacement and zero rotation at both ends (fixed supports).

### Loading Condition
The beam is subjected to a uniformly distributed gravity load along its length. The load per unit length is given by:
\[
q = - \rho A g
\]
Where:
- \( g = 9.81 \, \text{m/s}^2 \) (acceleration due to gravity).

---

Please use this information to perform the beam deflection analysis under the clamped-clamped boundary condition.


#1. Define Material & Beam Properties

In [ ]:
E = 200e9  # Young's modulus in Pascals (N/m²)
rho = 7800  # Density in kg/m³
g = 9.81    # Acceleration due to gravity (m/s²)
L = 1.0     # Total beam length in meters
b = 0.010   # Beam width in meters
h = 0.005   # Beam height in meters
A = b * h   # Cross-sectional area
I = (b * h**3) / 12  # Moment of inertia


#2. Define Finite Element Parameters

In [ ]:
N = 40       # Number of elements
Nnodes = N + 1  # Number of nodes
ell = L / N  # Length of each element


#3. Construct Element Stiffness Matrix for Beam

In [ ]:
K_e = (E * I / ell**3) * np.array([
    [12, 6 * ell, -12, 6 * ell],
    [6 * ell, 4 * ell**2, -6 * ell, 2 * ell**2],
    [-12, -6 * ell, 12, -6 * ell],
    [6 * ell, 2 * ell**2, -6 * ell, 4 * ell**2]
])


#4. Assemble Global Stiffness Matrix

In [ ]:
K = np.zeros((2 * Nnodes, 2 * Nnodes))

for i in range(N):
    K_local_indices = np.arange(2 * i, 2 * i + 4)
    K[np.ix_(K_local_indices, K_local_indices)] += K_e


#5. Apply Uniformly Distributed Load (UDL)

In [ ]:
q = -rho * A * g  # Load per unit length (N/m)
f = np.zeros((2 * Nnodes, 1))

q_el = np.array([q * ell / 2, q * ell**2 / 12, q * ell / 2, -q * ell**2 / 12]).reshape(4, 1)

for i in range(N):
    f_local_indices = np.arange(2 * i, 2 * i + 4)
    f[f_local_indices] += q_el


#6. Apply Boundary Conditions (Clamped-Clamped)

In [ ]:
bc_indices = [0, 1, 2 * Nnodes - 2, 2 * Nnodes - 1]  # Fixed DOFs

K = np.delete(K, bc_indices, axis=0)
K = np.delete(K, bc_indices, axis=1)
f = np.delete(f, bc_indices, axis=0)


#7. Solve for Displacements

In [ ]:
dx_vec = np.linalg.solve(K, f)


#8. Reconstruct Full Displacement Vector

In [ ]:
dx = np.zeros(Nnodes)
dtheta = np.zeros(Nnodes)

# Correct extraction: displacements are even indices, rotations are odd indices
displacements = dx_vec[::2, 0]   # Every even index (w1, w2, ..., w39)
rotations = dx_vec[1::2, 0]      # Every odd index (theta1, theta2, ..., theta39)

dx[1:-1] = displacements
dtheta[1:-1] = rotations

# Convert deflection to mm for better visualization
dx_mm = dx * 1000


#9. Plot Results

In [ ]:
x = np.linspace(0, L, Nnodes)  # Nodal positions

fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

ax[0].plot(x, np.zeros_like(x), 'k-', lw=3, label="Beam")
ax[0].quiver(x, np.zeros_like(x) + 0.002, np.zeros_like(x), np.full_like(x, -0.01),
             angles='xy', scale_units='xy', scale=1, color='r', width=0.003, headwidth=3, headlength=5)
ax[0].set_ylabel("Beam & Load")
ax[0].set_title("Clamped-Clamped Beam with Uniform Load")
ax[0].legend(loc='upper right')
ax[0].grid(True)
ax[0].set_ylim(-0.12, 0.05)  # Adjust to show arrows

ax[1].plot(x, dx_mm, 'b-', lw=2, label="Deflection (mm)")
ax[1].set_ylabel("Deflection [mm]")
ax[1].set_title("Deflected Shape")
ax[1].axhline(0, color='k', linestyle='--', lw=1)
ax[1].legend()
ax[1].grid(True)

ax[2].plot(x, dtheta, 'g-', lw=2, label="Slope (radians)")
ax[2].set_ylabel("Slope [radians]")
ax[2].set_xlabel("Beam Length [m]")
ax[2].set_title("Slope Along the Beam")
ax[2].axhline(0, color='k', linestyle='--', lw=1)
ax[2].legend()
ax[2].grid(True)

plt.tight_layout()
plt.show()
